<a href="https://colab.research.google.com/github/arnavt1605/Projects/blob/main/DL/MNIST_ConvNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
torch.manual_seed(0)

train_data, val_data = torch.utils.data.random_split(MNIST(root = 'data', train= True, download= True, transform= ToTensor()), [50000, 10000])
test_data = MNIST(root= "data", train= False, download=True, transform= ToTensor())

In [15]:
train_dataloader= DataLoader(train_data, shuffle= True, batch_size= 64)
val_dataloader= DataLoader(val_data, shuffle= False, batch_size= 64)
test_dataloader= DataLoader(test_data, shuffle=False, batch_size= 64)

In [16]:

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_stack(x)
        logits = self.classifier(x)
        return logits


In [17]:
for images, labels in train_dataloader:
    print(images.shape)
    break


torch.Size([64, 1, 28, 28])


In [18]:
model= CNN().to(device)

In [19]:
loss_function = nn.CrossEntropyLoss()

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

In [21]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [22]:
def validate(dataloader, model, loss_function):
  model.eval()
  total_loss= 0
  correct= 0
  total= 0
  with torch.no_grad():
    for image, label in dataloader:
      image= image.to(device)
      label= label.to(device)

      pred = model(image)
      loss= loss_function(pred, label)
      total_loss+= loss

      predicted_classes = pred.argmax(dim=1)
      correct+= (predicted_classes == label).sum().item()
      total+= label.size(0)

  avg_loss= total_loss / len(dataloader)
  accuracy= correct/total * 100
  print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")


In [23]:
def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)
            label= label.to(device)

            preds = model(image)
            loss = loss_function(preds, label)
            total_loss += loss.item()

            predicted_classes = preds.argmax(dim=1)
            correct += (predicted_classes == label).sum().item()
            total += label.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

In [24]:
epochs= 10
for epoch in range(epochs):
  print(f"Epoch {epoch+1} of {epochs}:")

  train(train_dataloader, model, loss_function, optimizer)
  validate(val_dataloader, model, loss_function)

test(test_dataloader, model, loss_function)


Epoch 1 of 10:
Training Average Loss: 0.2759
Validation Loss: 0.0810, Validation Accuracy: 97.54%
Epoch 2 of 10:
Training Average Loss: 0.0841
Validation Loss: 0.0647, Validation Accuracy: 98.06%
Epoch 3 of 10:
Training Average Loss: 0.0609
Validation Loss: 0.0424, Validation Accuracy: 98.85%
Epoch 4 of 10:
Training Average Loss: 0.0484
Validation Loss: 0.0425, Validation Accuracy: 98.69%
Epoch 5 of 10:
Training Average Loss: 0.0404
Validation Loss: 0.0379, Validation Accuracy: 98.85%
Epoch 6 of 10:
Training Average Loss: 0.0346
Validation Loss: 0.0385, Validation Accuracy: 98.89%
Epoch 7 of 10:
Training Average Loss: 0.0294
Validation Loss: 0.0344, Validation Accuracy: 99.02%
Epoch 8 of 10:
Training Average Loss: 0.0259
Validation Loss: 0.0368, Validation Accuracy: 98.95%
Epoch 9 of 10:
Training Average Loss: 0.0223
Validation Loss: 0.0380, Validation Accuracy: 98.91%
Epoch 10 of 10:
Training Average Loss: 0.0194
Validation Loss: 0.0319, Validation Accuracy: 99.15%
Test Loss: 0.0251, 